# RNN (III)

## <span style="font-size:25px">RNN Operation Principal</span>

<img src="picture/ML_img_37_1.png" alt="ML_img_37_1" width=873 height=327>

|     | <span style="font-size:15px">② LR</span> | <span style="font-size:15px">③ Summation</span> | <span style="font-size:15px">④ Output</span> | <span style="font-size:15px; color:#FE2E64">TensorFlow API</span> |
|:---:|:---:|:---:|:---:|:---:|
| <span style="font-size:15px">Hidden layer</span> | $\small{\text{A1}\cdot\text{W}_\text{ih} = \text{Z2}}$ | $\small{\text{Z2}+\text{H}_\text{cur}\cdot\text{W}_\text{hh}+\text{b}_\text{n} =\text{R2}}$ | $\small{\text{tanh(R2) = \textcolor{#FE2E64}{H}}_{\textcolor{#FE2E64}{\text{t}}}}$ | tf.contrib.rnn.BasicRNNCell(⋯) <br> tf.nn.dynamic_rnn(⋯) |

<br>

|     | <span style="font-size:15px">⑤ LR</span> | <span style="font-size:15px">⑥ Output</span> | loss function | <span style="font-size:15px; color:#FE2E64">TensorFlow API</span> |
|:---:|:---:|:---:|:---:|:---:|
| <span style="font-size:15px">Output layer</span> | $\small{\text{A2}\cdot\text{W}_\text{ho} = \text{Z3}}$ | $\small{\text{softmax(Z3 + b}_\text{o}) = \text{A3}}$ | Cross Entropy | tf.contrib.seq2seq.sequence_loss(⋯) |

<img src="picture/ML_img_37_2.png" alt="ML_img_37_2" width=873 height=200>
<br>
<img src="picture/ML_img_37_3.png" alt="ML_img_37_3" width=873 height=700>

## <span style="font-size:25px">e.g. ('gohome')</span>

<img src="picture/ML_img_37_4.png" alt="ML_img_37_4" width=873 height=700>

## <span style="font-size:25px">RNN operation (input_data gohom => label_data ohome)</span>

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np

tf.compat.v1.disable_eager_execution()
tfv1 = tf.compat.v1

# 'gohome' Data Creation
idx2char = ['g', 'o', 'h', 'm', 'e']    # g = 0, o = 1, h = 2, m = 3, e = 4

x_data = [[0, 1, 2, 1, 3]]     # gohome

x_one_hot = [[[1, 0, 0, 0],     # g 0
              [0, 1, 0, 0],     # o 1
              [0, 0, 1, 0],     # h 2
              [0, 1, 0, 0],     # o 1
              [0, 0, 0, 1]]]    # m 3

t_data = [[1, 2, 1, 3, 4]]      # ohome

mun_classes = 5      # correct answer size, so it's the size represented as one-hot
input_dim = 4        # one-hot size, 0~3 => total 4
hidden_size = 5      # output from the RNN, 5 to directly predict one-hot
batch_size = 1       # one sentence
sequence_length = 5  # input sentence length, gohom = 5
learning_rate = 1e-1

# placeholder(number point, [batch size, height, width, number of chennal])
X = tfv1.placeholder(tfv1.float32, [None, sequence_length, input_dim])
T = tfv1.placeholder(tfv1.float32, [None, sequence_length])

In [ ]:
cell = tfv1.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)

initial_state = cell.zero_state(batch_size, tf.float32)

outputs, _states = tfv1.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

In [3]:
weights = tfv1.ones([batch_size, sequence_length])

T = tf.cast(T, tf.int32)
seq_loss = tfa.seq2seq.sequence_loss(logits=outputs, targets=T, weights=weights)

loss = tfv1.reduce_mean(seq_loss)

train = tfv1.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [4]:
y = prediction = tfv1.argmax(outputs, axis=2)

with tfv1.Session() as sess:
    sess.run(tfv1.global_variables_initializer())
    for step in range(2001):
        loss_val = sess.run([loss, train], feed_dict={X: x_one_hot, T: t_data})
        result = sess.run(y, feed_dict={X: x_one_hot})
        
        if step % 400 == 0:
            print("step =", step, ", loss_val =", loss_val, ", prediction =", result, ", target =", t_data)
            result_str = [idx2char[c] for c in np.squeeze(result)]
            print("\tPrediction =", ''.join(result_str))

step = 0 , loss_val = [1.4958118, None] , prediction = [[0 2 1 3 4]] , target = [[1, 2, 1, 3, 4]]
	Prediction = ghome
step = 400 , loss_val = [0.44907007, None] , prediction = [[1 2 1 3 4]] , target = [[1, 2, 1, 3, 4]]
	Prediction = ohome
step = 800 , loss_val = [0.44365376, None] , prediction = [[1 2 1 3 4]] , target = [[1, 2, 1, 3, 4]]
	Prediction = ohome
step = 1200 , loss_val = [0.44144154, None] , prediction = [[1 2 1 3 4]] , target = [[1, 2, 1, 3, 4]]
	Prediction = ohome
step = 1600 , loss_val = [0.5217691, None] , prediction = [[1 2 4 3 4]] , target = [[1, 2, 1, 3, 4]]
	Prediction = oheme
step = 2000 , loss_val = [0.52174264, None] , prediction = [[1 2 4 3 4]] , target = [[1, 2, 1, 3, 4]]
	Prediction = oheme
